In [1]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import numpy as np



Using TensorFlow backend.


In [2]:
from keras import applications
def vgg16Model(image_size = (224, 224), if_draw_model = False):
    width = image_size[0]
    height = image_size[1]
    
    base_model = applications.VGG16(input_tensor=Input((height, width, 3)), weights='imagenet', include_top=False)

    for layers in base_model.layers:
        layers.trainable = True
        
    for i, layer in enumerate(base_model.layers):
        print(i, layer.name)
        
   # for layer in base_model.layers[25:]:
   #     layer.trainable = True

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(10, activation='sigmoid')(x)
    model = Model(base_model.input, x)
    
    if if_draw_model:
        SVG(model_to_dot(model).create(prog='dot', format='svg'))
    
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [3]:
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

def restNet_model(image_size = (224, 224), if_draw_model = False):

    width = image_size[0]
    height = image_size[1]
    
    base_model = ResNet50(input_tensor=Input((height, width, 3)), weights='imagenet', include_top=False)

    for layers in base_model.layers:
        layers.trainable = True
        
    for i, layer in enumerate(base_model.layers):
        print(i, layer.name)
        
   # for layer in base_model.layers[25:]:
   #     layer.trainable = True

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(10, activation='sigmoid')(x)
    model = Model(base_model.input, x)
    
    if if_draw_model:
        SVG(model_to_dot(model).create(prog='dot', format='svg'))
    
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input, decode_predictions
import numpy as np

def inceptionV3_model(image_size = (224, 224), if_draw_model = False):

    width = image_size[0]
    height = image_size[1]
    
    base_model = InceptionV3(input_tensor=Input((height, width, 3)), weights='imagenet', include_top=False)

    for layers in base_model.layers:
        layers.trainable = True
        
    for i, layer in enumerate(base_model.layers):
        print(i, layer.name)
        
   # for layer in base_model.layers[25:]:
   #     layer.trainable = True

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(10, activation='sigmoid')(x)
    model = Model(base_model.input, x)
    
    if if_draw_model:
        SVG(model_to_dot(model).create(prog='dot', format='svg'))
    
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [ ]:
#简单模型
def simple_model(time_len=1):
    ch, row, col = 3, 66, 200  # camera format
    model = Sequential()
    model.add(Lambda(lambda x: x/127.5 - 1.,
  #  model.add(Lambda(lambda x: x,
            input_shape=( row, col, ch),
            output_shape=( row, col,ch)))
    model.add(Convolution2D(3, 3, 3, subsample=(2, 2), border_mode="same"))
    model.add(ELU())
    
    model.add(Flatten())
    model.add(Dropout(.5))
    model.add(ELU())
    model.add(Dense(10))
#  model.add(Lambda(nor_output_1))
    sgd = optimizers.SGD(lr=0.00003, momentum=0.9, nesterov=True)
    model.compile(loss='mean_squared_error', optimizer=sgd)

      

    return model

In [ ]:
import os
import shutil
from sklearn.utils import shuffle
np.random.seed(2017)

def prepare_val():
    dirname = 'imgs/val'
    dirname_src = 'imgs/train/'
    if os.path.exists(dirname):
        return
    os.mkdir(dirname)
    sub_dirs = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
    for sub_dir in sub_dirs:
        address_des = dirname + '/' + sub_dir
        os.mkdir(address_des)
        
        address_src = dirname_src + '/' + sub_dir
        train_filenames = os.listdir(address_src)
        train_filenames = shuffle(train_filenames)
        for file_src in train_filenames[0: int(len(train_filenames) / 10)]:
            add_con_list = file_src.split('/')
            add_old = dirname_src + sub_dir + '/' + add_con_list[-1]
            add_new = dirname + '/' + sub_dir + '/' + add_con_list[-1]
         #   print(add_old)
         #   print(add_new)
            shutil.move(add_old, add_new)  
        
prepare_val()

In [4]:
import os
import shutil
import pandas as pd
import cv2
from sklearn.utils import shuffle
np.random.seed(2017)

def get_driver_list():
    driver_id_list = []
    df = pd.read_csv("driver_imgs_list.csv")
    for i in range(df.shape[0]):
        driver_id = df.loc[i]["subject"]
        is_saved_id = False
        for saved_id in driver_id_list:
            if saved_id == driver_id:
                is_saved_id = True
                break
        
        if is_saved_id == False:
            driver_id_list.append(driver_id)
    return driver_id_list

In [5]:

def prepare_data():
    dirbase = 'imgs/train2/'
    if os.path.exists(dirbase):
        return
    os.mkdir(dirbase)
        
    driver_id_list = get_driver_list()
    
    df = pd.read_csv("driver_imgs_list.csv")  
    sub_dirs = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
    for driver_id in driver_id_list:
        os.mkdir(dirbase + driver_id)
        for sub_dir in sub_dirs:
            os.mkdir(dirbase + driver_id + '/' + sub_dir)

    for i in range(df.shape[0]):
        driver_id = df.loc[i]["subject"]
        class_name = df.loc[i]["classname"]
        img = df.loc[i]["img"]
        add_old = 'imgs/train/' + class_name+'/' + img
        add_new = dirbase + driver_id + '/' + class_name + '/' + img
        if os.path.exists(add_old):
            shutil.move(add_old, add_new)
        
prepare_data()    

In [6]:
image_size=(224, 224)
batch_size =64

In [7]:
#训练
from keras.callbacks import *
from keras.models import *
#from keras.applications.vgg19 import preprocess_input
from keras.applications.resnet50 import preprocess_input, decode_predictions
#from keras.applications.inception_v3 import preprocess_input, decode_predictions

import copy
def get_file_num(driver_id):
    train_base_add = 'imgs/train2/'
    add_driver = train_base_add + driver_id
    sub_dirs = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
    num = 0
    for sub_dir in sub_dirs:
        add_driver_class = add_driver + '/' + sub_dir
        train_filenames = os.listdir(add_driver_class)
        num += len(train_filenames)
    return num

def train_model(Model):
    model = Model()
    
    train_datagen = ImageDataGenerator(
 #   shear_range=0.2,
 #   zoom_range=0.2,
    preprocessing_function=preprocess_input,
 #   horizontal_flip=True
    )
    driver_id_list = get_driver_list()
    train_base_add = 'imgs/train2/'
    
    
    patience = 5
    loss_list = []
    
    best_model = 0
    save_loss_callback = LambdaCallback(on_epoch_end=lambda epoch, logs:loss_list.append(logs['val_loss']))
                            
    for epoch in range(15):
        print('step---------------------- {%d}'%(epoch))
        for driver_id in driver_id_list:
            nb_train_samples = get_file_num(driver_id)
            
            train_generator = train_datagen.flow_from_directory(
                train_base_add + driver_id,
                target_size=(image_size[0], image_size[1]),
                batch_size=batch_size,
                class_mode='categorical')
    
            val_generator = train_datagen.flow_from_directory(
                'imgs/val',
                target_size=(image_size[0], image_size[1]),
                batch_size=batch_size,
                class_mode='categorical')
        
            model.fit_generator(
                train_generator,
                steps_per_epoch=nb_train_samples // batch_size,
                epochs=1,
                workers=6,
                callbacks=[save_loss_callback],
                validation_data = val_generator)
            
            last_loss = loss_list[-1]
            if_best_loss = True
            for loss in loss_list:
                if loss < last_loss:
                    if_best_loss = False
            if if_best_loss == True:
            #    best_model =copy.deepcopy(model) 
                if epoch >= 6:
                    print('last model loss ------------------------------------------%f'%(last_loss))
                    model.save('model.h5')
    
    
    model = load_model('model.h5')
    
    return model


In [17]:
from keras.callbacks import *
from keras.models import *
#from keras.applications.vgg19 import preprocess_input
from keras.applications.resnet50 import preprocess_input, decode_predictions
#from keras.applications.inception_v3 import preprocess_input, decode_predictions
from sklearn.preprocessing import OneHotEncoder
from skimage.io import imread, imsave
from scipy.misc import imresize
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

def get_file_num(driver_id):
    train_base_add = 'imgs/train2/'
    add_driver = train_base_add + driver_id
    sub_dirs = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
    num = 0
    for sub_dir in sub_dirs:
        add_driver_class = add_driver + '/' + sub_dir
        train_filenames = os.listdir(add_driver_class)
        num += len(train_filenames)
    return num

def load_image(path):
    img = imread(path)
    img = imresize(img, (image_size[1], image_size[0]))
    return img

def load_driver_imgs(driver_id, pre_add = 'imgs/train2/'):
    base_add = pre_add + driver_id
    sub_dirs = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
    
    img_list = []
    label_list = []
    for i, sub_dir in enumerate(sub_dirs) :
        img_class_add = base_add + '/' + sub_dir
        img_names = os.listdir(img_class_add)
        for img_name in img_names:
            
            img = load_image(img_class_add + '/' + img_name)
            img_list.append(img)
         #   label_val = [0,0,0,0,0,0,0,0,0,0]
         #   label_val[i] = 1
            label_list.append(sub_dir)
            
    encoder = LabelEncoder()
    encoder.fit(label_list)
    encoded_Y = encoder.transform(label_list)
# convert integers to dummy variables (i.e. one hot encoded)
    label_list = np_utils.to_categorical(encoded_Y)
    
    img_list = np.array(img_list)
    label_list = np.array(label_list)
    return img_list, label_list

def train_model_2(Model):
    model = Model()
    
    train_datagen = ImageDataGenerator(
 #   shear_range=0.2,
 #   zoom_range=0.2,
    preprocessing_function=preprocess_input,
 #   horizontal_flip=True
    )
    driver_id_list = get_driver_list()
    train_base_add = 'imgs/train2/'
    
    
    patience = 5
    loss_list = []
    
    best_model = model
    save_loss_callback = LambdaCallback(on_epoch_end=lambda epoch, logs:loss_list.append(logs['val_loss']))
    
    val_datagen = ImageDataGenerator(
 #   shear_range=0.2,
 #   zoom_range=0.2,
    preprocessing_function=preprocess_input,
 #   horizontal_flip=True
    )
    x_val, y_val = load_driver_imgs('val', 'imgs/')
    val_datagen.fit(x_val)
    
    
    for epoch in range(13):
        print('step---------------------- {%d}'%(epoch))
        for driver_id in driver_id_list:
            nb_train_samples = get_file_num(driver_id)
            
            x_train, y_train = load_driver_imgs(driver_id)
        
            train_datagen.fit(x_train)
            
            if epoch >= 5:
                model.fit_generator(
                    train_datagen.flow(x_train, y_train, batch_size=batch_size),
                    shuffle=True,
                    steps_per_epoch=nb_train_samples // batch_size,
                    epochs=1,
                    workers=6,
                    callbacks=[save_loss_callback],
                    validation_data = val_datagen.flow(x_val, y_val, batch_size=batch_size)               
                )
                last_loss = loss_list[-1]
                if_best_loss = True
                for loss in loss_list:
                    if loss < last_loss:
                        if_best_loss = False
                if if_best_loss == True:
                
                    best_model = model
                    print('last model loss ------------------------------------------%f'%(last_loss))
                    model.save('model.h5')
            else:
                model.fit_generator(
                train_datagen.flow(x_train, y_train, batch_size=batch_size),
                shuffle=True,
                steps_per_epoch=nb_train_samples // batch_size,
                epochs=1,
                workers=6,
                validation_data = val_datagen.flow(x_val, y_val, batch_size=batch_size))
    
    model = load_model('model.h5')
    return model


In [9]:
import pandas as pd
import cv2
def test_model(model, file_name):
    df = pd.read_csv("sample_submission.csv")
    
    test_address = 'imgs/test'
    test_filenames = os.listdir(test_address)
 #   test_filenames = test_filenames[:10]
    X = np.zeros((len(test_filenames), image_size[0], image_size[1], 3), dtype=np.uint8)

    for i, test_file in enumerate(test_filenames) :
        X[i] = cv2.resize(cv2.imread(test_address + '/' + test_file), image_size)
    
    y_pred = model.predict(X, batch_size=batch_size, verbose=0)
  #  print(y_pred)
    
    for i, fname in enumerate(test_filenames):
      #  print(fname)
        df.set_value(i, 'img', fname)
        sub_dirs = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
        for ii, sb in enumerate(sub_dirs):
            df.set_value(i, sb, y_pred[i][ii])


    df.to_csv(file_name, index=None)
    df.head(10)
    

In [10]:
import pandas as pd
import cv2
#from keras.applications.vgg19 import preprocess_input
#from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.applications.inception_v3 import preprocess_input, decode_predictions

def test_model_2(model, file_name):
    df = pd.read_csv("sample_submission.csv")
    
    test_address = 'imgs/test'

    
    test_datagen = ImageDataGenerator(
 #   shear_range=0.2,
 #   zoom_range=0.2,
    preprocessing_function=preprocess_input,
 #   horizontal_flip=True
    )
    test_generator = test_datagen.flow_from_directory(
        test_address,
        target_size=(image_size[0], image_size[1]),
        shuffle = "false",
        class_mode='categorical',
        batch_size=1)

    filenames = test_generator.filenames
    nb_samples = len(filenames)
   
    y_pred = model.predict_generator(test_generator, steps = nb_samples)
  #  print(y_pred)
    
    for i, fname in enumerate(filenames):
      #  print(fname)
        df.set_value(i, 'img', fname)
        sub_dirs = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
        for ii, sb in enumerate(sub_dirs):
            df.set_value(i, sb, y_pred[i][ii])


    df.to_csv(file_name, index=None)
    df.head(10)

In [ ]:
model = train_model_2(restNet_model)
test_model(model, 'pred.csv')

0 input_3
1 conv1
2 bn_conv1
3 activation_99
4 max_pooling2d_3
5 res2a_branch2a
6 bn2a_branch2a
7 activation_100
8 res2a_branch2b
9 bn2a_branch2b
10 activation_101
11 res2a_branch2c
12 res2a_branch1
13 bn2a_branch2c
14 bn2a_branch1
15 add_33
16 activation_102
17 res2b_branch2a
18 bn2b_branch2a
19 activation_103
20 res2b_branch2b
21 bn2b_branch2b
22 activation_104
23 res2b_branch2c
24 bn2b_branch2c
25 add_34
26 activation_105
27 res2c_branch2a
28 bn2c_branch2a
29 activation_106
30 res2c_branch2b
31 bn2c_branch2b
32 activation_107
33 res2c_branch2c
34 bn2c_branch2c
35 add_35
36 activation_108
37 res3a_branch2a
38 bn3a_branch2a
39 activation_109
40 res3a_branch2b
41 bn3a_branch2b
42 activation_110
43 res3a_branch2c
44 res3a_branch1
45 bn3a_branch2c
46 bn3a_branch1
47 add_36
48 activation_111
49 res3b_branch2a
50 bn3b_branch2a
51 activation_112
52 res3b_branch2b
53 bn3b_branch2b
54 activation_113
55 res3b_branch2c
56 bn3b_branch2c
57 add_37
58 activation_114
59 res3c_branch2a
60 bn3c_branc

/home/ubuntu/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:25: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


step---------------------- {0}
Epoch 1/1
10/10 [==============================] - 13s 1s/step - loss: 0.6712 - acc: 0.8441 - val_loss: 3.7846 - val_acc: 0.2620
Epoch 1/1
11/11 [==============================] - 8s 732ms/step - loss: 0.2804 - acc: 0.9199 - val_loss: 8.7061 - val_acc: 0.2758
Epoch 1/1
12/12 [==============================] - 8s 689ms/step - loss: 0.1844 - acc: 0.9568 - val_loss: 12.2395 - val_acc: 0.1547
Epoch 1/1
12/12 [==============================] - 8s 695ms/step - loss: 0.5523 - acc: 0.8920 - val_loss: 13.4705 - val_acc: 0.1542
Epoch 1/1
14/14 [==============================] - 9s 656ms/step - loss: 2.8053 - acc: 0.4489 - val_loss: 1.1921e-07 - val_acc: 0.1109
Epoch 1/1
17/17 [==============================] - 10s 589ms/step - loss: 1.1687 - acc: 0.6324 - val_loss: 13.0634 - val_acc: 0.1033
Epoch 1/1
17/17 [==============================] - 10s 590ms/step - loss: 1.7060 - acc: 0.5549 - val_loss: 13.2177 - val_acc: 0.1010
Epoch 1/1
17/17 [===========================

11/11 [==============================] - 8s 741ms/step - loss: 0.2181 - acc: 0.9345 - val_loss: 4.6163 - val_acc: 0.4747
Epoch 1/1
9/9 [==============================] - 8s 841ms/step - loss: 0.3820 - acc: 0.9132 - val_loss: 5.2609 - val_acc: 0.3397
Epoch 1/1
8/8 [==============================] - 7s 901ms/step - loss: 0.1858 - acc: 0.9492 - val_loss: 7.1044 - val_acc: 0.3076
Epoch 1/1
8/8 [==============================] - 7s 879ms/step - loss: 0.2193 - acc: 0.9460 - val_loss: 7.4140 - val_acc: 0.3478
Epoch 1/1
10/10 [==============================] - 8s 758ms/step - loss: 0.2449 - acc: 0.9391 - val_loss: 7.5698 - val_acc: 0.3335
Epoch 1/1
11/11 [==============================] - 8s 738ms/step - loss: 0.7045 - acc: 0.7434 - val_loss: 10.9682 - val_acc: 0.1851
Epoch 1/1
14/14 [==============================] - 9s 640ms/step - loss: 0.1381 - acc: 0.9630 - val_loss: 5.4502 - val_acc: 0.3947
Epoch 1/1
11/11 [==============================] - 8s 724ms/step - loss: 1.3588 - acc: 0.6084 - va

11/11 [==============================] - 8s 747ms/step - loss: 0.7654 - acc: 0.7727 - val_loss: 9.7484 - val_acc: 0.1404
Epoch 1/1
11/11 [==============================] - 8s 730ms/step - loss: 0.2146 - acc: 0.9456 - val_loss: 10.0870 - val_acc: 0.1730
Epoch 1/1
11/11 [==============================] - 8s 732ms/step - loss: 0.2001 - acc: 0.9345 - val_loss: 5.1849 - val_acc: 0.4189
Epoch 1/1
14/14 [==============================] - 9s 643ms/step - loss: 0.1595 - acc: 0.9522 - val_loss: 3.1306 - val_acc: 0.5306
Epoch 1/1
4/4 [==============================] - 6s 1s/step - loss: 0.2668 - acc: 0.9199 - val_loss: 2.2319 - val_acc: 0.5874
Epoch 1/1
11/11 [==============================] - 8s 733ms/step - loss: 0.1688 - acc: 0.9528 - val_loss: 0.9890 - val_acc: 0.7076
Epoch 1/1
11/11 [==============================] - 8s 733ms/step - loss: 0.0856 - acc: 0.9759 - val_loss: 0.8879 - val_acc: 0.7215
step---------------------- {5}
Epoch 1/1
10/10 [==============================] - 8s 766ms/step -

4/4 [==============================] - 6s 1s/step - loss: 0.2469 - acc: 0.9052 - val_loss: 0.8640 - val_acc: 0.7895
Epoch 1/1
11/11 [==============================] - 8s 732ms/step - loss: 0.1471 - acc: 0.9686 - val_loss: 0.9923 - val_acc: 0.7591
Epoch 1/1
11/11 [==============================] - 8s 732ms/step - loss: 0.1070 - acc: 0.9716 - val_loss: 0.6990 - val_acc: 0.8122
step---------------------- {7}
Epoch 1/1
10/10 [==============================] - 8s 764ms/step - loss: 0.0753 - acc: 0.9780 - val_loss: 0.6984 - val_acc: 0.8069
Epoch 1/1
11/11 [==============================] - 8s 736ms/step - loss: 0.1089 - acc: 0.9724 - val_loss: 0.7791 - val_acc: 0.7903
Epoch 1/1
12/12 [==============================] - 9s 709ms/step - loss: 0.4487 - acc: 0.8776 - val_loss: 6.7804 - val_acc: 0.3000
Epoch 1/1
12/12 [==============================] - 8s 697ms/step - loss: 0.2062 - acc: 0.9418 - val_loss: 4.9632 - val_acc: 0.3661
Epoch 1/1
14/14 [==============================] - 9s 654ms/step - 

In [ ]:
model = train_model(restNet_model)

In [ ]:
test_model(model, 'pred.csv')

In [ ]:
test_model_2(model, 'pred.csv')

In [ ]:
model = train_model(restNet_model)

In [ ]:
test_model(model, 'pred.csv')